# Load Packages

# Load Data

In [ ]:
using CSV
using DataFrames
using Gurobi
using Random
using Distances
using JuMP
using Gurobi

const GRB_ENV = Gurobi.Env(output_flag=1);

LoadError: ArgumentError: Package CSV not found in current path.
- Run `import Pkg; Pkg.add("CSV")` to install the CSV package.

# Base Model

In [ ]:
model = Model(Gurobi.Optimizer)

# Load data
pairs = CSV.read("./data/kidney_data_with_states.csv", DataFrame)
centers = CSV.read("./data/Transplant_Center_Dataset.csv", DataFrame)

# # Take random samples
pair_size = 2000
# center_size = 6
Random.seed!(42)
pairs = pairs[shuffle(1:nrow(pairs))[1:min(pair_size, nrow(pairs))], :]
# centers = centers[shuffle(1:nrow(centers))[1:min(center_size, nrow(centers))], :]

n = nrow(pairs)
m = nrow(centers)

println("Number of pairs: ", n)
println("Number of centers: ", m)

# Create model
model = Model(Gurobi.Optimizer)

# Decision variables
@variable(model, x[1:n, 1:n, 1:m], Bin)  # x[i,j,k] = 1 if donor i donates to patient j through center k
@variable(model, y[1:m], Bin)  # y[k] = 1 if center k is used

# Calculate distances
distances = Dict()
for i in 1:n, k in 1:m
    pair_coords = (pairs[i, :lat], pairs[i, :lng])
    center_coords = (centers[k, :lat], centers[k, :lng])
    distances[(i, k)] = haversine(pair_coords, center_coords, 6371)  # 6371 is Earth's radius in km
end

# Compatibility matrix
comp = Dict()
blood_type_matches = [
    ("O", "O"), ("O", "A"), ("O", "B"), ("O", "AB"),
    ("A", "A"), ("A", "AB"),
    ("B", "B"), ("B", "AB"),
    ("AB", "AB")
]

for i in 1:n, j in 1:n
    donor = pairs[i, :Donor]
    patient = pairs[j, :Patient]
    comp[(i, j)] = (donor, patient) in blood_type_matches ? 1 : 0
end

# Hyper Parameter
# Calculate d_max from the sampled data
d_max = 0.0
for i in 1:n, j in 1:n, k in 1:m
    # Calculate total distance for this potential pairing through center k
    total_distance = distances[(i,k)] + distances[(j,k)]
    global d_max = max(d_max, total_distance)
end

# Calculate alpha based on d_max and number of pairs
α = 0

println("Maximum distance in sampled data (d_max): ", round(d_max, digits=2), " km")
println("Calculated α: ", α)


# Base Model with Altrustic Donors

# Introducing Matching Score

# Introducing Transportation Cost

# Introducing Surgery Constrains